This tutorial based on:
https://sagemaker-examples.readthedocs.io/en/latest/hyperparameter_tuning/xgboost_direct_marketing/hpo_xgboost_direct_marketing_sagemaker_APIs.html

Modified by: Camilo Pestana

Objectives:
<ul>
<li>Be familiar with some common libraries for Data Analysis in Python such as Pandas.</li>
<li>Get familiar with AWS SageMaker to train Machine Learning models using the cloud</li>
<li>Get familiar with Jupyter Notebooks</li>
</ul>

# Install Missing libraries in your local computer

In [ ]:
# Install SageMaker
!pip install sagemaker
# Install pandas
!pip install pandas

# Create Role

NOTE: For security purposes we won't allow students to create Roles. However, you can see below what are the steps needed to create them.

1. Login to the AWS console -> IAM -> Roles -> Create Role

2. Create an IAM role and select the "SageMaker" service

3. Give the role "AmazonSageMakerFullAccess" permission

4. Review and create the role

5. The code needs to fetch the IAM role directly when the code is executed on a local machine. The code snippet to be used is given below:
```
    iam = boto3.client('iam')
    sagemaker_role = iam.get_role(RoleName='<sagemaker-IAM-role-name>')['Role']['Arn']
    ```

A Role has been created using the steps above and you can use it for your lab.
```
    iam = boto3.client('iam')
    sagemaker_role = iam.get_role(RoleName='Role_AWS_SageMaker')['Role']['Arn']
```

You need now to create a bucket or use an existing one and create inside the folders "sagemaker/STUDENTID-hpo-xgboost-dm"

# Prepare SageMaker session

In [ ]:
import sagemaker
import boto3

import numpy as np  # For matrix operations and numerical processing
import pandas as pd  # For munging tabular data
from time import gmtime, strftime
import os

region = 'ap-southeast-2'
smclient = boto3.Session().client("sagemaker")

iam = boto3.client('iam')
sagemaker_role = iam.get_role(RoleName='Role_AWS_SageMaker')['Role']['Arn']

student_id = "STUDENTID"
bucket = 'YOUR_BUCKET_NAME_HERE'
prefix = f"sagemaker/{student_id}-hpo-xgboost-dm"

# Download Dataset

Please take some time to read about the data with more detail [here](https://archive.ics.uci.edu/ml/datasets/bank+marketing)
Let’s start by downloading the direct marketing dataset from UCI’s ML Repository.

You can download the dataset manually or use the commands below. These commands should work for Linux and MacOS users.

In [ ]:
!wget -N https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip
!unzip -o bank-additional.zip

Now lets read this into a Pandas data frame and take a look at the data.

In [ ]:
data = pd.read_csv("./bank-additional/bank-additional-full.csv", sep=";")
pd.set_option("display.max_columns", 500)  # Make sure we can see all of the columns
pd.set_option("display.max_rows", 50)  # Keep the output on one page
data

Answer the following questions:
    - Which variables are categorical?
    - Which ones are numerical?

In [ ]:
data["no_previous_contact"] = np.where(
    data["pdays"] == 999, 1, 0
)  # Indicator variable to capture when pdays takes a value of 999
data["not_working"] = np.where(
    np.in1d(data["job"], ["student", "retired", "unemployed"]), 1, 0
)  # Indicator for individuals not actively employed
model_data = pd.get_dummies(data)  # Convert categorical variables to sets of indicators
model_data

Let’s remove the economic features and duration from our data as they would need to be forecasted with high precision to use as inputs in future predictions.

In [ ]:
model_data = model_data.drop(
    ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
    axis=1,
)

In [ ]:
model_data

# Split Data into training, validation and test

We’ll then split the dataset into training (70%), validation (20%), and test (10%) datasets and convert the datasets to the right format the algorithm expects. We will use training and validation datasets during training. Test dataset will be used to evaluate model performance after it is deployed to an endpoint.

Amazon SageMaker’s XGBoost algorithm expects data in the libSVM or CSV data format. For this lab, we’ll stick to CSV. Note that the first column must be the target variable and the CSV should not include headers. Also, notice that although repetitive it’s easier to do this after the train|validation|test split rather than before. This avoids any misalignment issues due to random reordering.

In [ ]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)

pd.concat([train_data["y_yes"], train_data.drop(["y_no", "y_yes"], axis=1)], axis=1).to_csv(
    "train.csv", index=False, header=False
)
pd.concat(
    [validation_data["y_yes"], validation_data.drop(["y_no", "y_yes"], axis=1)], axis=1
).to_csv("validation.csv", index=False, header=False)
pd.concat([test_data["y_yes"], test_data.drop(["y_no", "y_yes"], axis=1)], axis=1).to_csv(
    "test.csv", index=False, header=False
)

Now we’ll copy the file to S3 for Amazon SageMaker training to pickup.

In [ ]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation/validation.csv")
).upload_file("validation.csv")

# Setup Hyperparameter Optimization

In [ ]:
from time import gmtime, strftime, sleep

# Names have to be unique. You will get an error if you reuse the same name
tuning_job_name = f"{student_id}-xgboost-tuningjob-01"

print(tuning_job_name)

tuning_job_config = {
    "ParameterRanges": {
        "CategoricalParameterRanges": [],
        "ContinuousParameterRanges": [
            {
                "MaxValue": "1",
                "MinValue": "0",
                "Name": "eta",
            },
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "min_child_weight",
            },
            {
                "MaxValue": "2",
                "MinValue": "0",
                "Name": "alpha",
            },
        ],
        "IntegerParameterRanges": [
            {
                "MaxValue": "10",
                "MinValue": "1",
                "Name": "max_depth",
            }
        ],
    },
    "ResourceLimits": {"MaxNumberOfTrainingJobs": 2, "MaxParallelTrainingJobs": 2},
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {"MetricName": "validation:auc", "Type": "Maximize"},
}

In [ ]:
from sagemaker.image_uris import retrieve
# Use XGBoost algorithm for training
training_image = retrieve(framework="xgboost", region=region, version="latest")

s3_input_train = "s3://{}/{}/train".format(bucket, prefix)
s3_input_validation = "s3://{}/{}/validation/".format(bucket, prefix)

training_job_definition = {
    "AlgorithmSpecification": {"TrainingImage": training_image, "TrainingInputMode": "File"},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_train,
                }
            },
        },
        {
            "ChannelName": "validation",
            "CompressionType": "None",
            "ContentType": "csv",
            "DataSource": {
                "S3DataSource": {
                    "S3DataDistributionType": "FullyReplicated",
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_input_validation,
                }
            },
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/output".format(bucket, prefix)},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.m5.xlarge", "VolumeSizeInGB": 10},
    "RoleArn": sagemaker_role,
    "StaticHyperParameters": {
        "eval_metric": "auc",
        "num_round": "1",
        "objective": "binary:logistic",
        "rate_drop": "0.3",
        "tweedie_variance_power": "1.4",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 43200},
}

Now we can launch a hyperparameter tuning job by calling create_hyper_parameter_tuning_job API. After the hyperparameter tuning job is created, we can go to SageMaker console to track the progress of the hyperparameter tuning job until it is completed.

While Training you can take screenshots of the jobs you just launched on SageMaker-> Training -> 
Hyperparameter tuning jobs

In [ ]:
#Launch Hyperparameter Tuning Job
smclient.create_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name,
    HyperParameterTuningJobConfig=tuning_job_config,
    TrainingJobDefinition=training_job_definition,
)

Plese follow the steps from the following link and take screenshots to demonstrate you successfully completed your lab. 
[How to monitor the progress of a job?](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html)

You should also be able to check the output in s3://YOUR_BUCKET/sagemaker/STUDENTID-hpo-xgboost-dm/output

# End of Lab


1. In this lab we first used jupyter notebooks and pandas to explore a dataset.
2. Then we have learnt how to use boto3 and sagemaker to create training and hyperparameter optimization jobs, two important steps in every machine learning project.
3. After a job is learned SageMaker allows to deploy models in EC2 instances. However, this is out of the scope for this lab.

NOTE: Don't forget to remove all resources you used such as the bucket and data generated in S3.